In [1]:
!pip install spacy

In [2]:
import pandas as pd
import numpy as np
import nltk
import tensorflow as tf
import spacy
import string
from nltk import WordNetLemmatizer

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from googletrans import Translator

2025-05-28 11:47:10.318547: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748443630.331606   15971 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748443630.335555   15971 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748443630.346514   15971 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748443630.346529   15971 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748443630.346531   15971 computation_placer.cc:177] computation placer alr

In [3]:
df = pd.read_csv('../../data/data.csv')
df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


## Data Cleaning

In [4]:
#check info data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5842 entries, 0 to 5841
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentence   5842 non-null   object
 1   Sentiment  5842 non-null   object
dtypes: object(2)
memory usage: 91.4+ KB


In [5]:
# checkk null data
df.isnull().sum()

Sentence     0
Sentiment    0
dtype: int64

In [6]:
# check duplicated data
df.duplicated().sum()

np.int64(6)

In [7]:
# delete duplicate data
df.drop_duplicates(keep='first',inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5836 entries, 0 to 5841
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentence   5836 non-null   object
 1   Sentiment  5836 non-null   object
dtypes: object(2)
memory usage: 136.8+ KB


In [9]:
# Checking the unique values in Sentiment column
df.Sentiment.unique()

array(['positive', 'negative', 'neutral'], dtype=object)

In [10]:
df['Sentiment'].value_counts()

Sentiment
neutral     3124
positive    1852
negative     860
Name: count, dtype: int64

## Text Processing

In [11]:
# Function to preprocessing a single sentence
import re
def process_sentence(sentence):
    # Handle None or non-string input
    if not isinstance(sentence, str):
        return ""

    # Remove alphanumeric characters (numbers)
    sentence = re.sub(r'\d+', '', sentence)

    # Convert to lowercase
    sentence = sentence.lower()

    # Remove punctuation, except exclamation and question marks
    sentence = re.sub(r'[^\w\s!?]', '', sentence)

    # Remove stopwords
    stopwords = nltk.corpus.stopwords.words('english')
    words = sentence.split()
    filtered_words = [word for word in words if word not in stopwords]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    # Join words back into a sentence
    # return ' '.join(lemmatized_words)

    # Tokenization
    words = word_tokenize(sentence)

    return words

In [12]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/apo-pc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/apo-pc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
# Apply the process_sentence function to the Sentence column
df['Sentence'] = df['Sentence'].apply(process_sentence)

In [14]:
df.head()

,Sentence,Sentiment
0,"[the, geosolutions, technology, will, leverage...",positive
1,"[esi, on, lows, down, to, bk, a, real, possibi...",negative
2,"[for, the, last, quarter, of, componenta, s, n...",positive
3,"[according, to, the, finnishrussian, chamber, ...",neutral
4,"[the, swedish, buyout, firm, has, sold, its, r...",neutral


In [15]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [17]:
encoder = OneHotEncoder()
y = pd.DataFrame(encoder.fit_transform(df[['Sentiment']]).toarray())
y

,0,1,2
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0
...,...,...,...
5831,1.0,0.0,0.0
5832,0.0,1.0,0.0
5833,0.0,1.0,0.0
5834,0.0,1.0,0.0


In [18]:
# sorting feelings in the "Sentiment" column
label_mapping = {'negative':0,'positive':1,'neutral':2}
df['Sentiment'] = df['Sentiment'].replace(label_mapping)

/tmp/ipykernel_15971/1460423663.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Sentiment'] = df['Sentiment'].replace(label_mapping)


In [19]:
df.head()

,Sentence,Sentiment
0,"[the, geosolutions, technology, will, leverage...",1
1,"[esi, on, lows, down, to, bk, a, real, possibi...",0
2,"[for, the, last, quarter, of, componenta, s, n...",1
3,"[according, to, the, finnishrussian, chamber, ...",2
4,"[the, swedish, buyout, firm, has, sold, its, r...",2


In [24]:
# separation of data into training and testing sets
X = df['Sentence']
y = df['Sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,
                                                    random_state=42)

X_train = [' '.join(map(str, doc)) if isinstance(doc, list) else doc for doc in X_train]
X_test = [' '.join(doc) if isinstance(doc, list) else doc for doc in X_test]



In [25]:
# Vectorization of sentences
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

## Naive Bayes

In [26]:
# Train the Naive Bayes model
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf,y_train)

MultinomialNB()

In [27]:
# Evaluate model
y_pred = nb_classifier.predict(X_test_tfidf)
accuracy = accuracy_score(y_test,y_pred)

print(f'Accuracy: {accuracy:.2f}')
print(classification_report(y_test,y_pred))

Accuracy: 0.67
              precision    recall  f1-score   support

           0       0.83      0.04      0.07       280
           1       0.75      0.41      0.53       579
           2       0.66      0.98      0.79      1067

    accuracy                           0.67      1926
   macro avg       0.75      0.48      0.46      1926
weighted avg       0.71      0.67      0.61      1926



## Long-Short-Term-Memory

### Text normalization/tokenization

In [29]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix

In [30]:
num_words = 1000

tokenizer = Tokenizer(num_words=num_words,split=' ')
tokenizer.fit_on_texts(df['Sentence'].values)
x = tokenizer.texts_to_sequences(df['Sentence'].values)
x = pad_sequences(x)

In [48]:
# Define hyperparameters at the top for easy tuning
EMBEDDING_DIM = 128  # Increased from 32 for better representation
LSTM_UNITS = 128    # Increased from 64
DENSE_UNITS = 128   # Increased from 64
DROPOUT_RATE = 0.3  # Added dropout for regularization
L2_REGULARIZATION = 0.01  # Added L2 regularization
LEARNING_RATE = 0.001  # Explicit learning rate control

# Create more robust model
model = tf.keras.Sequential([
    # Embedding layer with mask_zero to handle variable length sequences
    tf.keras.layers.Embedding(num_words, EMBEDDING_DIM, mask_zero=True),

    # First Bidirectional LSTM layer with regularization
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(LSTM_UNITS,
                            return_sequences=True,
                            kernel_regularizer=tf.keras.regularizers.l2(L2_REGULARIZATION),
                            recurrent_regularizer=tf.keras.regularizers.l2(L2_REGULARIZATION))
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE),

    # Second Bidirectional LSTM layer
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(LSTM_UNITS//2,  # Reduced units for the second layer
                            kernel_regularizer=tf.keras.regularizers.l2(L2_REGULARIZATION))
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE),

    # Dense layers with regularization
    tf.keras.layers.Dense(DENSE_UNITS,
                         activation='relu',
                         kernel_regularizer=tf.keras.regularizers.l2(L2_REGULARIZATION)),
    tf.keras.layers.Dropout(DROPOUT_RATE),

    tf.keras.layers.BatchNormalization(),  # Added batch normalization

    # Output layer
    tf.keras.layers.Dense(3, activation='softmax')
])
# Build the model by providing input shape
model.build(input_shape=(None, x.shape[1]))
model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 42, 128)        │       128,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ (None, 42, 256)        │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 42, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ (None, 128)            │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 572,931 (2.19 MB)

 Trainable params: 572,675 (2.18 MB)

 Non-trainable params: 256 (1.00 KB)

In [53]:
# Compile with improved settings
# Custom learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=LEARNING_RATE,
    decay_steps=1000,
    decay_rate=0.9
)


model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='categorical_crossentropy',
    metrics=['accuracy',
             tf.keras.metrics.Precision(),
             tf.keras.metrics.Recall()]
)


In [54]:
y = pd.get_dummies(df['Sentiment']).values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,
                                                    random_state=42)

X_valid, x_test_val, y_valid, y_test_val = train_test_split(x_test,y_test,
                                                            test_size=0.5,
                                                            random_state=42)



In [55]:
early_stop = EarlyStopping(monitor='val_loss',patience=3)

In [63]:
# Define hyperparameters
EMBEDDING_DIM = 128
LSTM_UNITS = 128
DENSE_UNITS = 128
DROPOUT_RATE = 0.3
L2_REGULARIZATION = 0.01
LEARNING_RATE = 0.001

# Create model with cuDNN disabled for LSTM layers
model = tf.keras.Sequential([
    # Embedding layer without masking
    tf.keras.layers.Embedding(num_words, EMBEDDING_DIM),

    # First Bidirectional LSTM layer
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(LSTM_UNITS,
                            return_sequences=True,
                            kernel_regularizer=tf.keras.regularizers.l2(L2_REGULARIZATION),
                            use_bias=True,
                            use_cudnn=False)  # Disable cuDNN
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE),

    # Second Bidirectional LSTM layer
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(LSTM_UNITS//2,
                            kernel_regularizer=tf.keras.regularizers.l2(L2_REGULARIZATION),
                            use_bias=True,
                            use_cudnn=False)  # Disable cuDNN
    ),
    tf.keras.layers.Dropout(DROPOUT_RATE),

    # Dense layers
    tf.keras.layers.Dense(DENSE_UNITS,
                         activation='relu',
                         kernel_regularizer=tf.keras.regularizers.l2(L2_REGULARIZATION)),
    tf.keras.layers.Dropout(DROPOUT_RATE),

    tf.keras.layers.BatchNormalization(),

    # Output layer
    tf.keras.layers.Dense(3, activation='softmax')
])

# Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=LEARNING_RATE,
    decay_steps=1000,
    decay_rate=0.9
)

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='categorical_crossentropy',
    metrics=['accuracy',
             tf.keras.metrics.Precision(),
             tf.keras.metrics.Recall()]
)

# Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        mode='min'
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'best_model.h5',
        monitor='val_loss',
        save_best_only=True,
        mode='min',
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        min_lr=0.00001
    )
]

# Train the model
history = model.fit(
    x_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_data=(x_test_val, y_test_val),
    callbacks=callbacks
)

Epoch 1/20
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4822 - loss: 6.7749 - precision_2: 0.5403 - recall_2: 0.3108
Epoch 1: val_loss improved from inf to 1.74775, saving model to best_model.h5


146/146 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.4829 - loss: 6.7575 - precision_2: 0.5411 - recall_2: 0.3116 - val_accuracy: 0.5788 - val_loss: 1.7478 - val_precision_2: 0.5808 - val_recall_2: 0.5788 - learning_rate: 9.8474e-04
Epoch 2/20
141/146 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7040 - loss: 1.3255 - precision_2: 0.7599 - recall_2: 0.6397
Epoch 2: val_loss improved from 1.74775 to 1.07307, saving model to best_model.h5


146/146 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.7040 - loss: 1.3190 - precision_2: 0.7595 - recall_2: 0.6396 - val_accuracy: 0.6027 - val_loss: 1.0731 - val_precision_2: 0.6633 - val_recall_2: 0.5668 - learning_rate: 9.6970e-04
Epoch 3/20
145/146 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7477 - loss: 0.8033 - precision_2: 0.7829 - recall_2: 0.6941
Epoch 3: val_loss improved from 1.07307 to 0.82685, saving model to best_model.h5


146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7476 - loss: 0.8029 - precision_2: 0.7828 - recall_2: 0.6940 - val_accuracy: 0.7312 - val_loss: 0.8268 - val_precision_2: 0.7869 - val_recall_2: 0.6575 - learning_rate: 9.5490e-04
Epoch 4/20
143/146 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7666 - loss: 0.6295 - precision_2: 0.7998 - recall_2: 0.7302
Epoch 4: val_loss improved from 0.82685 to 0.71676, saving model to best_model.h5


146/146 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7663 - loss: 0.6298 - precision_2: 0.7995 - recall_2: 0.7298 - val_accuracy: 0.7295 - val_loss: 0.7168 - val_precision_2: 0.7637 - val_recall_2: 0.6918 - learning_rate: 9.4032e-04
Epoch 5/20
142/146 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7826 - loss: 0.5612 - precision_2: 0.8057 - recall_2: 0.7465
Epoch 5: val_loss did not improve from 0.71676
146/146 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.7821 - loss: 0.5622 - precision_2: 0.8053 - recall_2: 0.7461 - val_accuracy: 0.7209 - val_loss: 0.7576 - val_precision_2: 0.7356 - val_recall_2: 0.6764 - learning_rate: 9.2597e-04
Epoch 6/20
143/146 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7975 - loss: 0.5310 - precision_2: 0.8172 - recall_2: 0.7694
Epoch 6: val_loss did not improve from 0.71676
146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7973 - loss: 0.5314 - precision_2: 0.8171 - recall_2: 0.7691 - val_accuracy: 0.7312 - val_loss: 0.7219 - val_precision_2: 0

TypeError: This optimizer was created with a `LearningRateSchedule` object as its `learning_rate` constructor argument, hence its learning rate is not settable. If you need the learning rate to be settable, you should instantiate the optimizer with a float `learning_rate` argument.

In [64]:
loss, accuracy = history.evaluate(X_valid, y_valid, batch_size=32, verbose=2)

print(f'Loss on test data: {loss}')
print(f'Accuracy on test data: {accuracy}')

19/19 - 0s - 4ms/step - accuracy: 0.6182 - loss: 2.2241
Loss on test data: 2.224050760269165
Accuracy on test data: 0.6181507110595703


In [58]:
y_pred = history.predict(X_valid)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_valid, axis=1)

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


In [59]:
conf_matrix = confusion_matrix(y_test,y_pred)
print(conf_matrix)

[[ 28  19  41]
 [ 15 104  49]
 [ 54  30 244]]
